## 통합본

In [1]:
from langchain_community.document_loaders import PyMuPDFLoader
from langchain.document_loaders import PyPDFLoader
from langchain_community.document_loaders import UnstructuredPDFLoader
from langchain_community.document_loaders import PyPDFDirectoryLoader

from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_huggingface import HuggingFaceEmbeddings

from langchain_core.documents import Document

# from tqdm.autonotebook import tqdm, trange
from transformers import AutoTokenizer

import re
import os
from glob import glob
from datetime import datetime


def get_cur_time():
    return datetime.now().strftime('%Y-%m-%d %H:%M:%S')

# Embedding 모델 정의
# -----------------
# embeddings_model = HuggingFaceEmbeddings(
#     model_name="snunlp/KR-SBERT-V40K-klueNLI-augSTS",
#     model_kwargs={"device": "mps"},  # cpu : 'cpu', macOS: 'mps', CUDA: 'cuda'
#     # 모델이 CPU에서 실행되도록 설정. GPU를 사용할 수 있는 환경이라면 'cuda'로 설정할 수도 있음
#     encode_kwargs={
#         "normalize_embeddings": True
#     },  # 임베딩 정규화. 모든 벡터가 같은 범위의 값을 갖도록 함. 유사도 계산 시 일관성을 높여줌
# )

# 임베딩(Embedding) 정의
# embeddings_model = HuggingFaceEmbeddings(
#     model_name="BAAI/bge-m3",
#     model_kwargs={"device": "mps"},  # cpu : 'cpu', macOS: 'mps', CUDA: 'cuda'
#     # 모델이 CPU에서 실행되도록 설정. GPU를 사용할 수 있는 환경이라면 'cuda'로 설정할 수도 있음
#     encode_kwargs={
#         "normalize_embeddings": True
#     },  # 임베딩 정규화. 모든 벡터가 같은 범위의 값을 갖도록 함. 유사도 계산 시 일관성을 높여줌
# )
model_name_path = '../embedding_model/BAAI_bge-m3'
embeddings_model = HuggingFaceEmbeddings(
        model_name=model_name_path, 
        model_kwargs={"device": "mps"},  # cpu : 'cpu', macOS: 'mps', CUDA: 'cuda'
        # 모델이 CPU에서 실행되도록 설정. GPU를 사용할 수 있는 환경이라면 'cuda'로 설정할 수도 있음
        encode_kwargs={
            "normalize_embeddings": True
        },  # 임베딩 정규화. 모든 벡터가 같은 범위의 값을 갖도록 함. 유사도 계산 시 일관성을 높여줌
    )



# 1단계 : 임베딩할 파일 지정 및 PDF를 텍스트 문서로 변환
# -------------------------------------------
print("-" * 70)
print(f"[{get_cur_time()}] 1단계 : 임베딩할 파일 지정 및 텍스트 문서로 변환")
print("-" * 70)
cur_date = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
# 지정한 폴더 하위의 모든 pdf, PDF 파일 조회
# ----------------------------------
# DOCUMENT_FOLDER = "./data/jbb"
DOCUMENT_FOLDER = "./rag_data/jbb"
# DOCUMENT_FOLDER = "./data/jbb/신입행원연수_2023"
# DOCUMENT_FOLDER = "./data/jbb/책임자고시_2024"

file_lists = []
def search(dirname):
    try:
        filenames = os.listdir(dirname)
        for filename in filenames:
            full_filename = os.path.join(dirname, filename)
            if os.path.isdir(full_filename):
                search(full_filename)
            else:
                ext = os.path.splitext(full_filename)[-1]
                if ext == '.pdf' or ext == '.PDF': 
                    file_lists.append(full_filename)
    except PermissionError:
        pass

    return file_lists

pdf_files = search(DOCUMENT_FOLDER)
pdf_files_backup = pdf_files.copy()

pdf_file_tot_cnt = 0
pdf_file_tot_page_cnt = 0
pdf_docs = []

# pdf 파일 로딩
# -----------
for pdf_file_cnt, pdf_file in enumerate(pdf_files):
    # loader = PyMuPDFLoader(pdf_file)
    loader = PyPDFLoader(pdf_file)

    pdf_doc = loader.load()
    pdf_docs.extend(pdf_doc)

    # print(f"CNT: {pdf_file_cnt+1:>3}, 페이지 수: {len(pdf_doc):>3}, 파일명: {pdf_file}")
    pdf_file_tot_page_cnt += len(pdf_doc)
    pdf_file_tot_cnt += 1

print("-" * 70)
print(f"[{get_cur_time()}] 파일의 전체 갯수 : {pdf_file_tot_cnt}, 파일의 전체 페이지 수: {pdf_file_tot_page_cnt}")
print("-" * 70)


# 2단계 : 토큰 수를 기준으로 청크 단위로 Split
# ------------------------------------
# HuggingFace Embedding 모델의 Tokenizer를 사용하여 토큰화
# tokenizer = AutoTokenizer.from_pretrained("snunlp/KR-SBERT-V40K-klueNLI-augSTS")
# tokenizer = AutoTokenizer.from_pretrained("BAAI/bge-m3")
tokenizer = AutoTokenizer.from_pretrained(model_name_path)
# model_name_path = '../embedding_model/BAAI_bge-m3'

text_splitter = RecursiveCharacterTextSplitter.from_huggingface_tokenizer(
    tokenizer=tokenizer,
    chunk_size=300,
    chunk_overlap=0,
)

split_docs = text_splitter.split_documents(pdf_docs)
print(f"[{get_cur_time()}] PDF 문서의 페이지 수: {len(pdf_docs)}, 분할한 문서의 수: {len(split_docs)}")


## 일부 필요한 텍스트를 분할된 문서에 추가 및 불필요한 문자 삭제
final_docs = []

# 문서만 보여주기 위한 Document 객체 생성 및 저장
contents = ['보여줘', '알려줘', '']
for pdf_file_cnt, pdf_file in enumerate(pdf_files_backup):
    for content in contents:
        title = os.path.splitext(pdf_file)[0].split('/')[-1]    # 파일명만 추출
        doc = Document(page_content = f"전북은행 {title} {content}")
        doc.metadata['title'] = title        
        doc.metadata['source'] = pdf_file
        doc.metadata['page'] = 0
        
        final_docs.append(doc)

doc_cnt = len(final_docs)   # 문서만 보여주기위한 문서 수 


for doc in split_docs:
    title = os.path.splitext(doc.metadata['source'])[0].split('/')[-1]
    doc.page_content = (
        re.sub(r"(?<!\.)\n", " ", doc.page_content)
        + f"\n\n문서 : 전북은행 {title}"        
    )
    doc.metadata['title'] = title
    final_docs.append(doc)

print(f"[{get_cur_time()}] 분할한 문서의 수: {len(split_docs)}, 보여주기위한 문서수: {doc_cnt}, 처리할 총 문서의 수: {len(final_docs)}")

print(f"[{get_cur_time()}] 벡터 스토어 저장 Start ...")


# MAXIMUN BATCH SIZE(41666) 초과에 대한 대응 코드
# -------------------------------------------
## ValueError: Batch size 56732 exceeds maximum batch size 41666
## --------------------------------------------------------------
def split_list(input_list, chunk_size):
    for i in range(0, len(input_list), chunk_size):
        yield input_list[i:i + chunk_size]
        
# split_docs_chunked = split_list(split_docs, 10000)
split_docs_chunked = split_list(final_docs, 10000)

proc_tot_cnt = 0
for split_docs_chunk in split_docs_chunked:
    # Chroma DB 에 저장    
    vectorstore = Chroma.from_documents(        # 10000개씩 Chroma DB 에 저장
        documents=split_docs_chunk,
        embedding=embeddings_model,
        persist_directory="./Chroma_db/chroma_bank_law_db",
        # persist_directory="./chroma_bank_law_db",
        collection_name="bank_law_case",
    )
    proc_tot_cnt += len(split_docs_chunk)
    print(f"[{get_cur_time()}] Chroma DB 에 저장된 총 건수: {proc_tot_cnt}")    

print(f"[{get_cur_time()}] 문서 임베딩이 완료 되었습니다")

----------------------------------------------------------------------
[2024-08-22 11:21:47] 1단계 : 임베딩할 파일 지정 및 텍스트 문서로 변환
----------------------------------------------------------------------
----------------------------------------------------------------------
[2024-08-22 11:24:41] 파일의 전체 갯수 : 463, 파일의 전체 페이지 수: 6122
----------------------------------------------------------------------
[2024-08-22 11:24:59] PDF 문서의 페이지 수: 6122, 분할한 문서의 수: 17196
[2024-08-22 11:24:59] 분할한 문서의 수: 17196, 보여주기위한 문서수: 1389, 처리할 총 문서의 수: 18585
[2024-08-22 11:24:59] 벡터 스토어 저장 Start ...
[2024-08-22 11:27:20] Chroma DB 에 저장된 총 건수: 10000
[2024-08-22 11:29:34] Chroma DB 에 저장된 총 건수: 18585
[2024-08-22 11:29:34] 문서 임베딩이 완료 되었습니다


In [24]:
from langchain_community.vectorstores import Chroma
from langchain_huggingface import HuggingFaceEmbeddings

model_name_path = '../embedding_model/BAAI_bge-m3'
embeddings_model = HuggingFaceEmbeddings(
        model_name=model_name_path, 
        model_kwargs={"device": "mps"},  # cpu : 'cpu', macOS: 'mps', CUDA: 'cuda'
        # 모델이 CPU에서 실행되도록 설정. GPU를 사용할 수 있는 환경이라면 'cuda'로 설정할 수도 있음
        encode_kwargs={
            "normalize_embeddings": True
        },  # 임베딩 정규화. 모든 벡터가 같은 범위의 값을 갖도록 함. 유사도 계산 시 일관성을 높여줌
    )

# vectorstore = Chroma.from_documents(documents=final_docs,
#                                     embedding=embeddings_model,
#                                     collection_name="bank_law_case",
#                                     persist_directory="./chroma_bank_law_db")

# vectorstore = Chroma(persist_directory="./chroma_bank_law_db", embedding_function=embeddings_model, collection_name="bank_law_case")
vectorstore = Chroma(
    persist_directory="./Chroma_db/chroma_bank_law_db", embedding_function=embeddings_model, collection_name="bank_law_case",
)

In [35]:
# chroma_docs = vectorstore.similarity_search("예금 상품의 이자계산에 대해서 알려줘", k=5)
# chroma_docs = vectorstore.similarity_search("대여금고 취급 지침에 의하면 대여금고 취급할 경우 주의 사항에 대해서 알려줘", k=5)
# chroma_docs = vectorstore.similarity_search("A1003 이사회 규정에서 정하는 이사의 권한과 의무에 대해서 알려줘", k=3)
# chroma_docs = vectorstore.similarity_search("A1003 이사회 규정에서 의장을 선임은 어떻게 해", k=3)
# chroma_docs = vectorstore.similarity_search("A1003 이사회 규정에서 의장을 선임은 어떻게 해", k=3)
# chroma_docs = vectorstore.similarity_search("전북은행 전산운영위원회지침 에서 IT투자심의 건당5천만원 알려줘.", k=3)
# chroma_docs = vectorstore.similarity_search("문서 전산운영위원회 지침에서 정하는 에서 정하는 IT투자심의 및 사후관리 건당5천만원 대해서 알려주세요.", k=3)
# chroma_docs = vectorstore.similarity_search("문서 전산운영위원회지침의 건당 5천만원 IT투자심의가 뭐야", k=3)
# chroma_docs = vectorstore.similarity_search("전산운영위원회 지침 보여줘", k=3)
# chroma_docs = vectorstore.similarity_search("복지 규정의 문화체육 활동비 내용을 알려줘", k=3)
# chroma_docs = vectorstore.similarity_search_with_score("r2045 가계자금대출 업무지침 을 보여줘", k=1)
# chroma_docs = vectorstore.similarity_search_with_score("일반 직원본봉표 보여줘", k=5)
# chroma_docs = vectorstore.similarity_search_with_score("일반 직원본봉표 보여줘", k=2)
chroma_docs = vectorstore.similarity_search("일반 직원본봉표 보여줘", k=2)
# chroma_docs = vectorstore.similarity_search("자체 보안성 검토 절차", k=3)

# for doc, score in chroma_docs:
#     print(f'[ 페이지 내용 ] {len(doc.page_content)} score: {score}\n {doc.page_content}')
#     print(f'[ 메타 데이터 ] \n {str(doc.metadata["source"])} {str(int(doc.metadata["page"]) + 1)}')
#     print("-" * 80)

for doc in chroma_docs:
    print(f'[ 페이지 내용 ] {len(doc.page_content)} \n {doc.page_content}')
    print(f'[ 메타 데이터 ] \n {str(doc.metadata["source"])} {str(int(doc.metadata["page"]) + 1)}')
    print("-" * 80)    

[ 페이지 내용 ] 39 
 전북은행 별표 1 일반직원 본봉표 (2024. 7. 1. 개정) 알려줘
[ 메타 데이터 ] 
 ./rag_data/jbb/규정/별표 1 일반직원 본봉표 (2024. 7. 1. 개정).pdf 1
--------------------------------------------------------------------------------
[ 페이지 내용 ] 547 
 <별표 1> 일반직원 본봉표 (2024. 7. 1. 개정) 일반직원 본봉표 (단위 : 원)    급별  호봉7급 6급 5급 4급 3급 2급 1급 1 782,100 1,000,000 1,540,500 2,129,200 2,635,000 3,087,200 3,515,500  2 815,800 1,041,300 1,585,300 2,180,200 2,684,700 3,136,600 3,563,700  3 849,400 1,082,600 1,630,100 2,231,400 2,734,500 3,186,000 3,612,100  4 883,200 1,123,800 1,674,900 2,282,500 2,784,200 3,235,300 3,660,300  5 916,900 1,165,100 1,719,700 2,333,700 2,820,100 3,284,800 3,708,600  6 950,600 1,206,400 1,764,500 2,384,800 2,869,600 3,334,200 3,756,800

문서 : 전북은행 별표 1 일반직원 본봉표 (2024. 7. 1. 개정)
[ 메타 데이터 ] 
 ./rag_data/jbb/규정/별표 1 일반직원 본봉표 (2024. 7. 1. 개정).pdf 1
--------------------------------------------------------------------------------


In [9]:
chroma_docs = vectorstore.similarity_search_with_score("주식규정에 대해서 알려줘", k=2)

for doc, score in chroma_docs:
    print(f'[ 메타 데이터 ] \n {str(doc.metadata["source"])} {str(int(doc.metadata["page"]) + 1)}')
    print("-" * 80)
    url_params = {'source': doc.metadata["source"], 'page': doc.metadata["page"]}
    print(url_params)

[ 메타 데이터 ] 
 ./data/jbb/규정/(A1010) 주식규정.pdf 1
--------------------------------------------------------------------------------
{'source': './data/jbb/규정/(A1010) 주식규정.pdf', 'page': 0}
[ 메타 데이터 ] 
 ./data/jbb/규정/(I1002) 내부통제 규정 [개정(7) 2023. 3.15].pdf 8
--------------------------------------------------------------------------------
{'source': './data/jbb/규정/(I1002) 내부통제 규정 [개정(7) 2023. 3.15].pdf', 'page': 7}


## 파일이름의 "_" 를 제거

In [ ]:
import os

def remove_underscores_from_filenames(directory):
    try:
        # List all files in the specified directory
        for filename in os.listdir(directory):
            # Check if the filename contains underscores
            if '_' in filename:
                # Create the new filename by removing underscores
                new_filename = filename.replace('_', '')
                # Construct the full old and new file paths
                old_file = os.path.join(directory, filename)
                new_file = os.path.join(directory, new_filename)
                # Rename the file
                os.rename(old_file, new_file)
                print(f'Renamed: {old_file} to {new_file}')
        print("All files have been renamed.")
    except Exception as e:
        print(f"An error occurred: {e}")

# Example usage
# directory_path = '/Users/netager/prod_llm/Jabis/source/data/jbb/규정'
# directory_path = '/Users/netager/prod_llm/Jabis/source/data/jbb/신입행원연수_2023'
# directory_path = '/Users/netager/prod_llm/Jabis/source/data/jbb/책임자고시_2024'
directory_path = '/Users/netager/prod_llm/Jabis/source/data/jbb/IT업무매뉴얼'

remove_underscores_from_filenames(directory_path)

In [ ]:
final_docs[0:5]

In [ ]:
final_docs[0:5]

In [ ]:
# vectorstore = Chroma.from_documents(documents=final_docs,
#                                     embedding=embeddings_model,
#                                     collection_name="bank_law_case",
#                                     persist_directory="./chroma_bank_law_db")

# vectorstore = Chroma(persist_directory="./chroma_bank_law_db", embedding_function=embeddings_model, collection_name="bank_law_case")
vectorstore = Chroma(
    persist_directory="./chroma_bank_law_db", embedding_function=embeddings_model, collection_name="bank_law_case",
)

# chroma_docs = vectorstore.similarity_search("예금 상품의 이자계산에 대해서 알려줘", k=5)
# chroma_docs = vectorstore.similarity_search("대여금고 취급 지침에 의하면 대여금고 취급할 경우 주의 사항에 대해서 알려줘", k=5)
chroma_docs = vectorstore.similarity_search("A1003 이사회 규정에서 정하는 이사의 권한과 의무에 대해서 알려줘", k=3)
chroma_docs = vectorstore.similarity_search("A1003 이사회 규정에서 의장을 선임은 어떻게 해", k=3)
chroma_docs = vectorstore.similarity_search("A1003 이사회 규정에서 의장을 선임은 어떻게 해", k=3)
chroma_docs = vectorstore.similarity_search("전북은행 전산운영위원회지침 에서 IT투자심의 건당5천만원 알려줘.", k=3)
# chroma_docs = vectorstore.similarity_search("문서 전산운영위원회 지침에서 정하는 에서 정하는 IT투자심의 및 사후관리 건당5천만원 대해서 알려주세요.", k=3)
# chroma_docs = vectorstore.similarity_search("문서 전산운영위원회지침의 건당 5천만원 IT투자심의가 뭐야", k=3)
chroma_docs = vectorstore.similarity_search("자체 보안성 검토 절차", k=3)
for doc in chroma_docs:
    print(f'[ 페이지 내용 ] \n {doc.page_content}')
    print(f'[ 메타 데이터 ] \n {str(doc.metadata["source"])} {str(int(doc.metadata["page"]) + 1)}')
    print("-" * 70)

In [ ]:
from langchain_community.document_loaders import PyMuPDFLoader
from langchain.document_loaders import PyPDFLoader
from langchain_community.document_loaders import UnstructuredPDFLoader
from langchain_community.document_loaders import PyPDFDirectoryLoader

from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_huggingface import HuggingFaceEmbeddings

# from tqdm.autonotebook import tqdm, trange
from transformers import AutoTokenizer

import re
import os
from glob import glob

# Embedding 모델 정의
# -----------------
embeddings_model = HuggingFaceEmbeddings(
    model_name="snunlp/KR-SBERT-V40K-klueNLI-augSTS",
    model_kwargs={"device": "mps"},  # cpu : 'cpu', macOS: 'mps', CUDA: 'cuda'
    # 모델이 CPU에서 실행되도록 설정. GPU를 사용할 수 있는 환경이라면 'cuda'로 설정할 수도 있음
    encode_kwargs={
        "normalize_embeddings": True
    },  # 임베딩 정규화. 모든 벡터가 같은 범위의 값을 갖도록 함. 유사도 계산 시 일관성을 높여줌
)


# 1단계 : 임베딩할 파일 지정 및 PDF를 텍스트 문서로 변환
# -------------------------------------------
print("-" * 60)
print("1단계 : 임베딩할 파일 지정 및 텍스트 문서로 변환")
print("-" * 60)

# PyMuPDFLoader
# -------------
# loader = PyMuPDFLoader("../data/SPRI_AI_Brief_2023년12월호_F.pdf")
# loader = PyMuPDFLoader("../data/전자금융감독규정(금융위원회고시)(제2022-44호)(20230101).pdf")

# PyPDFLoader
# -----------
# loader = PyPDFLoader("../data/SPRI_AI_Brief_2023년12월호_F.pdf")
# loader = PyPDFLoader("../data/전자금융감독규정(금융위원회고시)(제2022-44호)(20230101).pdf")

# UnstructuredPDFLoader
# ---------------------
# loader = UnstructuredPDFLoader("../data/SPRI_AI_Brief_2023년12월호_F.pdf")
# loader = UnstructuredPDFLoader("../data/전자금융감독규정(금융위원회고시)(제2022-44호)(20230101).pdf")

# PyPDFDirectoryLoader
# --------------------
# loader = PyPDFDirectoryLoader("./data/fss/")
# loader = PyPDFDirectoryLoader("./data/jbb/매뉴얼/")
# docs = loader.load()

# pdf_files = glob(os.path.join('./data/fss', '*.pdf'))
# pdf_files = glob(os.path.join('./data/jbb/규정', '*.pdf'))
# pdf_files = glob(os.path.join('./data/jbb/지침', '*.pdf'))
# pdf_files = glob(os.path.join("./data/jbb/매뉴얼", "*.pdf"))
# pdf_files = glob(os.path.join("./data/jbb/신입행원연수_2023", "*.pdf"))
pdf_files = glob(os.path.join("./data/jbb/책임자고시_2024", "*.pdf"))


pdf_file_tot_cnt = 0
pdf_file_tot_page_cnt = 0
pdf_docs = []

for pdf_file_cnt, pdf_file in enumerate(pdf_files):
    loader = PyPDFLoader(pdf_file)
    # loader = PyMuPDFLoader(pdf_file)
    pdf_doc = loader.load()
    pdf_docs.extend(pdf_doc)

    print(f"CNT: {pdf_file_cnt+1:>3}, 페이지 수: {len(pdf_doc):>3}, 파일명: {pdf_file}")
    pdf_file_tot_page_cnt += len(pdf_doc)
    pdf_file_tot_cnt += 1

print("-" * 60)
print(
    f"파일의 전체 갯수 : {pdf_file_tot_cnt}, 파일의 전체 페이지 수: {pdf_file_tot_page_cnt}"
)
print("-" * 60)


# 2단계 : 토큰 수를 기준으로 청크 단위로 Split
# ------------------------------------
# HuggingFace Embedding 모델의 Tokenizer를 사용하여 토큰화
tokenizer = AutoTokenizer.from_pretrained("snunlp/KR-SBERT-V40K-klueNLI-augSTS")

text_splitter = RecursiveCharacterTextSplitter.from_huggingface_tokenizer(
    tokenizer=tokenizer,
    chunk_size=400,
    chunk_overlap=40,
)

split_docs = text_splitter.split_documents(pdf_docs)
print(f"PDF 문서의 페이지 수: {len(pdf_docs)}, 분할한 문서의 수: {len(split_docs)}")


## 일부 필요한 텍스트를 분할된 문서에 추가 및 불필요한 문자 삭제
final_docs = []

for doc in split_docs:
    doc.page_content = (
       f"### 이 문서는 '{doc.metadata['source']}' 파일의 {int(doc.metadata['page'])+1} 페이지에 있습니다.\n\n"
        + re.sub(r"(?<!\.)\n", " ", doc.page_content)
        # re.sub(r"(?<!\.)\n", " ", doc.page_content)
    )
    final_docs.append(doc)


# 벡터 스토어 저장
#collection_name="bank_law_case",
# vectorstore = Chroma.from_documents(
#     documents=final_docs,
#     embedding=embeddings_model,
#     persist_directory="./chroma_bank_law_db",
# )

print(f"문서 임베딩이 완료 되었습니다")

In [ ]:
retriever = vectorstore.as_retriever(
    search_kwargs={"k": 5},
)

query = "삼성에서 개발한 AI의 이름은?"
# query = f"그룹공동 데이터베이스의 구축 및 운영시 암호화에 대해서 알려주세요."
# query = f"전산운영위원회에서 정하는 심의 조정에 대해서 알려주세요."

retrieved_docs = retriever.invoke(query)

for doc in retrieved_docs:
    print(
        str(doc.metadata["source"]), str(doc.metadata["page"]), doc.page_content[:200]
    )
    print("-" * 40)

In [ ]:
from langchain_community.chat_models import ChatOllama
from langchain.prompts import ChatPromptTemplate

# Prompt
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

# RAG Chain
llm = ChatOllama(model="qwen2-7b-instruct-q8:latest", temperature=0)

from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough


def format_docs(docs):
    return "\n\n".join([d.page_content for d in docs])


# query = f"인공지능의 미래 또는 발전 방향에 대해서 알려주세요."
# query = f"삼성전자가 개발한 인공지능 AI의 이름은?"
# query = f"고객정보의 제공 및 관리규정에서 말하는 개인신용정보에 대해서 알려주세요"
# query = f"그룹공동 데이터베이스의 구축 및 운영시 암호화에 대해서 알려주세요."

retriever = vectorstore.as_retriever(
    search_kwagrs={
        "k": 2,
    }
)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

response = rag_chain.stream(query)
for res in response:
    print(res, end="", flush=True)

## 분할본

### Library Import

In [ ]:
from langchain_community.document_loaders import PyMuPDFLoader
from langchain.document_loaders import PyPDFLoader
from langchain_community.document_loaders import UnstructuredPDFLoader
from langchain_community.document_loaders import PyPDFDirectoryLoader

from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_huggingface import HuggingFaceEmbeddings
from tqdm.autonotebook import tqdm, trange

### 단계 1: 문서 로드(Load Documents)

In [ ]:
# 단계 1: 문서 로드(Load Documents)

# PyMuPDFLoader
# -------------
# loader = PyMuPDFLoader("../data/SPRI_AI_Brief_2023년12월호_F.pdf")
# loader = PyMuPDFLoader("../data/전자금융감독규정(금융위원회고시)(제2022-44호)(20230101).pdf")

# PyPDFLoader
# -----------
# loader = PyPDFLoader("../data/SPRI_AI_Brief_2023년12월호_F.pdf")
# loader = PyPDFLoader("../data/전자금융감독규정(금융위원회고시)(제2022-44호)(20230101).pdf")

# UnstructuredPDFLoader
# ---------------------
# loader = UnstructuredPDFLoader("../data/SPRI_AI_Brief_2023년12월호_F.pdf")
# loader = UnstructuredPDFLoader("../data/전자금융감독규정(금융위원회고시)(제2022-44호)(20230101).pdf")

# PyPDFDirectoryLoader
# --------------------
# loader = PyPDFDirectoryLoader("./data/fss/")
# loader = PyPDFDirectoryLoader("./data/jbb/매뉴얼/")
# docs = loader.load()

# Multi PyMuPDF Loader
# ---------------------
import os
from glob import glob

# pdf_files = glob(os.path.join('./data/jbb/지침', '*.pdf'))
# pdf_files = glob(os.path.join('./data/fss', '*.pdf'))
pdf_files = glob(os.path.join("./data/jbb/매뉴얼", "*.pdf"))

print("-" * 60)
print("1단계 : 임베딩할 파일 지정 및 로딩")
print("-" * 60)

pdf_file_tot_cnt = 0
for pdf_file_cnt, pdf_file in enumerate(pdf_files):
    print(f"파일 No: {pdf_file_cnt+1:>3}, 파일명: {pdf_file}")
    pdf_file_tot_cnt += 1
print("-" * 60)
print(f"파일의 전체 갯수 : {pdf_file_tot_cnt}")
print("-" * 60)
# print(f"문서의 수: {len(docs)}\n")
# print("[메타데이터]\n")
# print(docs[010].metadata)
# print("\n========= [앞부분] 미리보기 =========\n")
# print(docs[10].page_content[:50])

In [ ]:
# pdf 파일을 읽어서 텍스트로 변환
from langchain_community.document_loaders import PyMuPDFLoader

pdf_docs = []
for pdf_file in pdf_files:
    loader = PyMuPDFLoader(pdf_file)
    pdf_doc = loader.load()
    pdf_docs.extend(pdf_doc)

    print(f"페이지 수: {len(pdf_doc)}, 파일명: {pdf_file}")

# print(f"문서의 수: {len(pdf_docs)}\n")  # 문서의 페이지 수
# print("[메타데이터]\n")
# print(pdf_docs[0].metadata)
# print("\n========= [앞부분] 미리보기 =========\n")
# print(data[0].page_content[:50])

### Embedding 정의

In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings

# embeddings = HuggingFaceEmbeddings(
#     model_name="BAAI/bge-m3",
#     model_kwargs={"device": "mps"},  # cpu : 'cpu', macOS: 'mps', CUDA: 'cuda'
#     # 모델이 CPU에서 실행되도록 설정. GPU를 사용할 수 있는 환경이라면 'cuda'로 설정할 수도 있음
#     encode_kwargs={
#         "normalize_embeddings": True
#     },  # 임베딩 정규화. 모든 벡터가 같은 범위의 값을 갖도록 함. 유사도 계산 시 일관성을 높여줌
# )

embeddings_model = HuggingFaceEmbeddings(
    model_name="snunlp/KR-SBERT-V40K-klueNLI-augSTS",
    model_kwargs={"device": "mps"},  # cpu : 'cpu', macOS: 'mps', CUDA: 'cuda'
    # 모델이 CPU에서 실행되도록 설정. GPU를 사용할 수 있는 환경이라면 'cuda'로 설정할 수도 있음
    encode_kwargs={
        "normalize_embeddings": True
    },  # 임베딩 정규화. 모든 벡터가 같은 범위의 값을 갖도록 함. 유사도 계산 시 일관성을 높여줌
)

In [ ]:
embeddings = embeddings_model.embed_documents(
    [
        "온라인 쇼핑몰에서 주문한 제품이 불량품으로 배송되었습니다. 이에 대한 법적 책임은 누구에게 있나요?",
        "구입한 전자제품이 고장나서 환불을 요청했지만 거부당했습니다. 피해 보상을 받을 수 있나요?",
        "호텔 예약 후 도착했는데 예약이 취소되었다고 했습니다. 이에 대한 대응 방법은 무었인가요?",
        "자동차 수리 후 동일한 문제가 재발했습니다. 수리업체에 대한 법적 조치를 취할 수 있나요?",
        "항공편이 지연되어 중요한 일정을 놓쳤습니다. 이에 대한 피해 보상을 받을 수 있나요?",
    ]
)
len(embeddings), len(embeddings[0])

In [ ]:
embedded_query = embeddings_model.embed_query(
    "에어컨 제품 불량에 대해서 보상 받을 수 있을까요?"
)
embedded_query[:5]

### Chunking

In [ ]:
# 단계 2: 문서 분할(Split Documents)
# Token 수를 기준으로 문서를 청크 단위로 분할
# HuggingFace Embedding 모델의 Tokenizer를 사용하여 토큰화
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("snunlp/KR-SBERT-V40K-klueNLI-augSTS")

from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter.from_huggingface_tokenizer(
    tokenizer=tokenizer,
    chunk_size=200,
    chunk_overlap=20,
)
pdf_docs = data
split_docs = text_splitter.split_documents(pdf_docs)
print("pdf 문서수: ", len(pdf_docs), "split 문서수: ", len(split_docs))
# print(split_docs[0].page_content)
# type(split_docs[0])
total = 0
for i in range(5):
    print(
        f"{i} 문서의 문자수 : {len(split_docs[i].page_content)} 토큰 갯수 : {len(tokenizer.encode(split_docs[i].page_content))}"
    )
    total = i + 1
    if total > 5:
        break

In [ ]:
split_docs[0].metadata

### Indexing

In [ ]:
import re

final_docs = []

for doc in split_docs:
    doc.page_content = (
        f"### 이 문서는 '{doc.metadata['source']}' 파일의 {int(doc.metadata['page'])+1} 페이지에 있습니다.\n\n"
        + re.sub(r"(?<!\.)\n", " ", doc.page_content)
    )
    final_docs.append(doc)

print(final_docs[0].page_content)

### Chroma DB 정의

In [ ]:
from langchain_community.vectorstores import Chroma

vectorstore = Chroma.from_documents(
    documents=final_docs,
    embedding=embeddings_model,
    collection_name="bank_law_case",
    persist_directory="./chroma_bank_law_db",
)

In [ ]:
# chroma_docs = vectorstore.similarity_search("삼성에서 개발한 AI의 이름은?", k=5)
# chroma_docs = vectorstore.similarity_search("전산운영위원회에서 정하는 IT투자심의 및 사후관리 건당5천만원 대해서 알려주세요.", k=5)
chroma_docs = vectorstore.similarity_search("삼성전자가 개발한 AI의 이름은?", k=1)
for doc in chroma_docs:
    print(
        str(doc.metadata["source"]),
        str(int(doc.metadata["page"]) + 1),
        doc.page_content[:200],
    )
    print("-" * 40)

### Retrieval

In [ ]:
retriever = vectorstore.as_retriever(
    search_kwargs={"k": 5},
)

# query = "삼성에서 개발한 AI의 이름은?"
# query = f"그룹공동 데이터베이스의 구축 및 운영시 암호화에 대해서 알려주세요."
query = f"전산운영위원회에서 정하는 심의 조정에 대해서 알려주세요."

retrieved_docs = retriever.invoke(query)

for doc in retrieved_docs:
    print(
        str(doc.metadata["source"]), str(doc.metadata["page"]), doc.page_content[:200]
    )
    print("-" * 40)

### Generateion

In [ ]:
from langchain_community.chat_models import ChatOllama
from langchain.prompts import ChatPromptTemplate

# Prompt
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)
prompt

### RAG 생성 및 실행

In [ ]:
# RAG Chain
llm = ChatOllama(model="qwen2-7b-instruct-q8:latest", temperature=0)

from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough


def format_docs(docs):
    return "\n\n".join([d.page_content for d in docs])


# query = f"인공지능의 미래 또는 발전 방향에 대해서 알려주세요."
query = f"삼성전자가 개발한 인공지능 AI의 이름은?"
# query = f"고객정보의 제공 및 관리규정에서 말하는 개인신용정보에 대해서 알려주세요"
# query = f"그룹공동 데이터베이스의 구축 및 운영시 암호화에 대해서 알려주세요."

retriever = vectorstore.as_retriever(
    search_kwagrs={
        "k": 2,
    }
)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

response = rag_chain.stream(query)
for res in response:
    print(res, end="", flush=True)

In [ ]:
import os

# 기존 사진 백업 필수!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
# 기존에 있는 사진들을 삭제됨

# 기존 파일 경로
folder_path = "./data/jbb/매뉴얼"

# 새로 저장 경로 (정확히 맞아야 함)
dst_path = "./data/jbb/매뉴얼"

# 폴더안에있는 파일들 이름 리스트로 가져오기
file_names = os.listdir(folder_path)

print(file_names)
# 확인

for file in file_names:
    f_name, f_format = file.split(sep=".P")  # 파일 이름, 파일 확장자 split
    # print(f_name, f_format)

    src_file = os.path.join(folder_path, file)
    dest_file = os.path.join(folder_path, f_name + ".pdf")

    print("rename" + file + " to " + dest_file)
    os.rename(src_file, dest_file)